This demo uses simulated annealing to solve the traveling sales man problem with 100 nodes,
the program runs 10 times and save the result into the result folder, please check the result folder to see the results.

In [1]:
import numpy as np
import networkx as nx
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
from tqdm import tqdm
from memory_profiler import profile
import time

In [2]:
class TSP():
    def __init__(self,coords):
        self.coords = coords
        self.start = 0
        self.length = len(coords)
        self.solution = []
    def initial_s(self):
        l = self.length
        s_init = np.arange(l)
        start = np.random.randint(0,l)
        self.start = start
        temp = s_init[0]
        s_init[0] = s_init[start]
        s_init[start] = temp
        self.solution = s_init
        return s_init
    @staticmethod
    def get_dis(a,b):
        return np.linalg.norm(a-b)
    def roulette(self,s):
        p1,p2 = np.random.choice(np.arange(1,len(s)),2,replace=False)
        if p1>p2:
            p1,p2 = p2,p1
        p =2 
        if p == 2:
            #insert
            l=s.tolist()
            l.insert(p2+1,l[p1])
            del l[p1]
            s = np.array(l)
        return s
    def total_dis(self,s):
        dist = 0
        for i,j in zip(s,s[1:]):
            dist+= self.get_dis(self.coords[i],self.coords[j])
        dist +=self.get_dis(self.coords[s[-1]],self.coords[s[0]])
        return dist

In [3]:
def simulated_annealing(tsp,fix_start=False,save=False):
    T = 100
    cooling_rate = 0.95
    if fix_start==False:
        s_current = tsp.initial_s()
    else:
        s_current = tsp.solution
    coords = tsp.coords[s_current]
    cost_current = tsp.total_dis(s_current)
    t_list = []
    cost_list=[]
    t_list.append(T)
    cost_list.append(cost_current)
    #subplot 2 , the route before SA
    fig= plt.figure(figsize=(10,5))
    fig.suptitle('Solving Traveling SalesMan with SA ,100 Cities')
    ax1= fig.add_subplot(132)
    ax2= fig.add_subplot(133)
    for first, second in zip(coords,coords[1:]):
        ax1.plot([first[0],second[0]],[first[1],second[1]],'b')
    ax1.plot([coords[-1][0],coords[0][0]],[coords[-1][1],coords[0][1]],'b')
    ax1.plot(coords[0][0],coords[0][1],'go')
    for c in coords[1:]:
        ax1.plot(c[0],c[1],'ro')
    #begin SA
    while T > 1:
        for i in range (500):
            s_new = tsp.roulette(s_current)
            cost_new = tsp.total_dis(s_new)
            if cost_new < cost_current:
                cost_current = cost_new
                s_current = s_new
                coords = tsp.coords[s_current]
            else :
                x=np.random.uniform()
                if x< np.exp(-(cost_new-cost_current)/T):
                    cost_current = cost_new
                    s_current = s_new
                    coords = tsp.coords[s_current]
        T = T*cooling_rate
        t_list.append(T)
        cost_list.append(cost_current)
    # subplot 3 , the route after SA
    for first, second in zip(coords,coords[1:]):
        ax2.plot([first[0],second[0]],[first[1],second[1]],'b')
    ax2.plot([coords[-1][0],coords[0][0]],[coords[-1][1],coords[0][1]],'b')
    ax2.plot(coords[0][0],coords[0][1],'go')
    for c in coords[1:]:
        ax2.plot(c[0],c[1],'ro')
    #subplot 1 , the cost curve
    ax3= fig.add_subplot(131)
    ax3.plot(t_list,cost_list,'c')
    ax3.set_title('Final Cost : '+str(cost_current))
    ax3.set_xlabel('Temperature')
    ax3.set_ylabel('Cost')
    ax3.invert_xaxis()
    #if fig.show() then save , the plot is blank
    if save==False:
        fig.show()

    return s_current.tolist(),cost_current,fig

In [4]:
%load_ext memory_profiler

In [6]:
if __name__ =='__main__':
    tsp_coords =np.loadtxt('./tsp_points.csv',delimiter=',')
    tsp = TSP(tsp_coords)
    tsp.initial_s()
    cost_list=[]
    time_list=[]
    for i in tqdm(range(10)):
        start_time = time.time()
        %memit s,c,fig =simulated_annealing(tsp,fix_start=True,save=True)
        end_time = time.time()
        time_list.append(end_time-start_time)
        cost_list.append(c)
        fig.savefig('./result/TSP/SA'+str(i+1)+'.png',dpi=fig.dpi)
        fig.clf()
        with open('./result/TSP/SA_result.txt','a+') as f:
            f.write('solution'+str(i+1)+': '+str(s)+'\n')
            f.write('cost'+str(i+1)+':'+str(c)+'\n')
            f.write('\n')
    with open('./result/TSP/SA_result.txt','a+') as f:
        f.write('max time :'+str(np.max(time_list))+'s\n')
        f.write('min time :'+str(np.min(time_list))+'s\n')
        f.write('mean time :'+str(np.mean(time_list))+'s\n')
        f.write('max cost :'+str(np.max(cost_list))+'\n')
        f.write('min cost :'+str(np.min(cost_list))+'\n')
        f.write('mean cost :'+str(np.mean(cost_list))+'\n')

  0%|          | 0/10 [00:00<?, ?it/s]

peak memory: 126.30 MiB, increment: 6.20 MiB


 10%|█         | 1/10 [00:20<03:04, 20.54s/it]

peak memory: 130.37 MiB, increment: 1.03 MiB


 20%|██        | 2/10 [00:40<02:43, 20.43s/it]

...

 80%|████████  | 8/10 [02:43<00:40, 20.41s/it]

peak memory: 146.96 MiB, increment: 0.00 MiB


 90%|█████████ | 9/10 [03:03<00:20, 20.33s/it]

peak memory: 149.17 MiB, increment: 0.00 MiB


100%|██████████| 10/10 [03:24<00:00, 20.41s/it]
